In [2]:
from dotenv import find_dotenv, load_dotenv

load_dotenv((find_dotenv()))

True

🟢 Part A — LCEL (|) Chain
Build a tone-safe reply generator using: prompt | llm | StrOutputParser()

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [9]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
system_message_persona = SystemMessagePromptTemplate.from_template('Act as a Customer Executive Expert in {domain} space, your resolutions are short and polite in tone.')
human_query = HumanMessagePromptTemplate.from_template('Tell me how can I debug an {issue} issue.')

message = [system_message_persona, human_query]

template = ChatPromptTemplate(message)

chain1 = template | llm | StrOutputParser()

output = chain1.invoke({
    'domain': 'Plumbing',
    'issue':'Roof Leakage'
})

print(output)

To debug a roof leakage issue, follow these steps:

1. **Visual Inspection**: Check for obvious signs of damage, such as missing shingles, cracks, or rusted flashings.
2. **Check Gutters and Downspouts**: Ensure they are clear of debris to avoid water backup.
3. **Interior Inspection**: Look for water stains or mold on ceilings and walls that might indicate the leak's location.
4. **Roof Slope**: Verify that your roof has the proper slope to direct water away from the home.
5. **Search for Blisters or Bubbles**: These can indicate a problem under the surface of roofing materials.
6. **Use Water Test**: Have someone inside the home watch for leaks while you pour water over suspected areas on the roof.

If the issue persists, it’s best to consult a professional roofer for a thorough evaluation.
